In [1]:
# This notebook is an implementatio of an optical CNOT operation using PBSs. Based on: https://doi.org/10.1103/PhysRevA.64.062311
import qutip as qt
from swap_TN_direct import *

import qutip as qt
import numpy as np
from quimb.tensor.tensor_arbgeom import tensor_network_apply_op_vec

/home/asingal/home/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/asingal/home/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
def quantum_encoder(mean_photon_num, N, psi_control, control_indices, error_tolerance):
    """This function performs the quantum encoder, basically "copy" one state into two modes. Obviously, you aren't copying shit. You are simply entangling another 
    state (modes (0,1) or mode b in the paper) with the control state, creating a "copy". The bell state required to make the copy is added before the control MPS 
    ((0,1):(H,V){b,d} and (2,3):(H,V){a,c}). {a,c} modes are measured out at the end of the function and hence, only 2 modes are pre-added to the retured MPS.   
    
    """
    # Entangled state from EPS
    vacuum = create_vacuum_state(num_modes=8, N=N)
    bell_state, _ = light_source(vacuum, N, mean_photon_num, 8, error_tolerance, compress=True, contract=True)

    psi = extend_MPS(bell_state, psi_control)

    # PBS op: (The V mode is transmitted and not reflected)
    U_PBS_V = create_BS_MPO(site1 = 3, site2 = bell_state.L+control_indices[1], theta=np.pi/2, total_sites = psi.L, N = N, tag = r"$PBS$")
    enforce_1d_like(U_PBS_V, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_PBS_V, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Measuring D_d
    # This is meant to change the basis from HV -> FS: (See https://doi.org/10.1103/PhysRevA.64.062311)
    U_PBS_FS = create_BS_MPO(site1 = 2, site2 = 3, theta=np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_PBS_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_PBS_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Performing measurements:
    BSM_POVM_1_OPs = generate_sqrt_POVM_MPO(sites=[2], outcome = 1, total_sites=psi.L, efficiency=1, N=N, pnr = True)
    BSM_POVM_1_OPs.extend(generate_sqrt_POVM_MPO(sites=[3], outcome = 0, total_sites=psi.L, efficiency=1, N=N, pnr = True))

    for POVM_OP in BSM_POVM_1_OPs:
        psi = tensor_network_apply_op_vec(POVM_OP, psi, compress=True, contract = True, cutoff = error_tolerance)

    return psi


def destructive_CNOT(control_b_sites, target_sites, psi, N, error_tolerance):

    # print("control_b_sites:", control_b_sites)
    # print("target_sites", target_sites)

    # Rotaing bases of encoded control's b mode and the target mode:
    U_rotator_FS = create_BS_MPO(site1 = control_b_sites[0], site2 = control_b_sites[1], theta=np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    U_rotator_FS = create_BS_MPO(site1 = target_sites[0], site2 = target_sites[1], theta=np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Applying PBS in rotated basis (only the V modes are reflected and the H modes are transmitted. Hence, the V modes undergo pi/2 rotation and the H modes undergo no rotation):
    
    # Implementation using SWAP operator. This does not generalize to higher truncations. 
    # SWAP = qt.qip.operations.swap().full()
    # U_PBS_F = create_MPO(1, 3, psi.L, SWAP, N, r"$PBS$")
    U_PBS_F = create_BS_MPO(site1 = target_sites[1], site2 = control_b_sites[1], theta=np.pi/2, total_sites = psi.L, N = N, tag = r"$PBS$")
    enforce_1d_like(U_PBS_F, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_PBS_F, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Undoing rotations:
    U_inverse_rotator_FS = create_BS_MPO(site1 = target_sites[0], site2 = target_sites[1], theta=-np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_inverse_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_inverse_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    U_inverse_rotator_FS = create_BS_MPO(site1 = control_b_sites[0], site2 = control_b_sites[1], theta=-np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_inverse_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_inverse_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Measuring the b mode (name d after the PBS)
    BSM_POVM_1_OPs = generate_sqrt_POVM_MPO(sites=[control_b_sites[0]], outcome = 1, total_sites=psi.L, efficiency=1, N=N, pnr = True)
    BSM_POVM_1_OPs.extend(generate_sqrt_POVM_MPO(sites=[control_b_sites[1]], outcome = 0, total_sites=psi.L, efficiency=1, N=N, pnr = True))

    for POVM_OP in BSM_POVM_1_OPs:
        psi = tensor_network_apply_op_vec(POVM_OP, psi, compress=True, contract = True, cutoff = error_tolerance)

    return psi

def CNOT(psi_control_modes, psi_target_modes, psi_control, psi_target, N, mean_photon_num, error_tolerance):
    """Pass psi_target as None if the same MPS has both the control and target modes.
    Args:
        psi_control_modes (list): List of control modes (H,V).
        psi_target_modes (list): List of target modes (H,V).
        psi_control (MPS): MPS for the control modes.
        psi_target (MPS): MPS for the target modes, can be None if the same MPS as target is used.
        N (int): Number of photons.
        mean_photon_num (float): Mean photon number for the EPS used in implemeting the CNOT gate.
        error_tolerance (float): Tolerance for numerical errors in tensor network operations.
    Returns:
        MPS: The resulting MPS after applying the CNOT operation.
    """

    psi_encoded_control = quantum_encoder(mean_photon_num, N, psi_control, psi_control_modes, error_tolerance)

    # read_quantum_state(psi_encoded_control,  N, num_states = 6)

    if not psi_target == None:
        psi = extend_MPS(psi_target, psi_encoded_control)
        psi_control_b_modes = [psi_target.L, psi_target.L+1] # [site + psi_target.L for site in psi_control_modes]
    
    else: 
        psi_control_b_modes = [0,1]
        psi_target_modes = [4+site for site in psi_target_modes] # We add 4 since the additional modes from the EPS are pre-added to the MPS. 
        psi = psi_encoded_control

    psi = destructive_CNOT(psi_control_b_modes, psi_target_modes, psi, N, error_tolerance)

    # read_quantum_state(psi, N, num_states = 6)

    norm = psi.normalize()
    for _ in range(4):
        psi.measure(0, remove = True, renorm = True, inplace = True)
    psi[-1].modify(data=psi[-1].data * norm**0.5)

    return psi


In [3]:
N = 2
mean_photon_num = 0.05
error_tolerance = 1e-10

In [4]:
# Defining the control and target inputs here
a_dag = qt.create(N).full()
I = qt.identity(N).full()

a_H = np.kron(a_dag, I)
a_V = np.kron(I, a_dag)

# Testing the case with two MPSs for control and target
vacuum = create_vacuum_state(num_modes=2, N=N)

# control_operator = 1/np.sqrt(3) * a_H - np.sqrt(2/3) * a_V
control_operator = a_V
control_MPO = create_MPO(0, 1, 2, control_operator, N, "control")
psi_control = tensor_network_apply_op_vec(control_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

# Our target is in the H state 
target_operator = a_V
target_MPO = create_MPO(0, 1, 2, target_operator, N, "target")
psi_target = tensor_network_apply_op_vec(target_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)


# Testing the case with only one MPS for both control and target
vacuum = create_vacuum_state(num_modes=8, N=N)

# control_operator = 1/np.sqrt(3) * a_H + np.sqrt(2/3) * a_V
control_operator = a_H
control_MPO = create_MPO(2, 3, 8, control_operator, N, "control")
psi_control = tensor_network_apply_op_vec(control_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

# Our target is in the H state 
target_operator = 1/np.sqrt(3) * a_H + np.sqrt(2/3) * a_V
target_MPO = create_MPO(6, 7, 8, target_operator, N, "target")
psi_combined = tensor_network_apply_op_vec(target_MPO, psi_control, compress=True, contract = True, cutoff = error_tolerance)

In [5]:
psi = CNOT((2,3), (6,7), psi_combined, None, N, mean_photon_num, error_tolerance)

# psi = quantum_encoder(mean_photon_num, N, psi_combined, (2,3), error_tolerance)

read_quantum_state(psi, N, num_states=4)

unitary_BS [[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]
unitary_BS [[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]
unitary_BS [[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]
unitary_BS [[ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]]
unitary_BS [[ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1

In [ ]:
vacuum = create_vacuum_state(num_modes=2, N=N)

# control_operator = 1/np.sqrt(3) * a_H - np.sqrt(2/3) * a_V
operator = a_V
MPO = create_MPO(0, 1, 2, operator, N, "control")
psi = tensor_network_apply_op_vec(MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

print("state:\n", psi.to_dense())

read_quantum_state(psi, N, num_states=1)

# H = create_BS_MPO_try(site1 = 0, site2 = 1, theta=np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
# H = generalized_mode_mixer(0, 1, np.pi/4, [np.pi/2,-np.pi/2], [np.pi,-np.pi], [-np.pi,np.pi], 2, N)

# This definition is based on the paper: https://arxiv.org/pdf/quant-ph/9706022
H = generalized_mode_mixer(0, 1, -np.pi/4, [0,-np.pi], [0,-np.pi], [0,0], 2, N)
# H = generalized_mode_mixer(0, 1, np.pi/4, 0, 0, 0, 2, N)
enforce_1d_like(H, site_tags=psi.site_tags, inplace=True)
psi = tensor_network_apply_op_vec(H, psi, compress=True, contract = True, cutoff = error_tolerance)

read_quantum_state(psi, N, num_states=1)

state:
 [[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]]
Corresponding Basis terms:
0H1V_A - 1 - [1.+0.j]
unitary_BS
 [[1.    +0.j     0.    +0.j     0.    +0.j     0.    +0.j    ]
 [0.    +0.j     0.7071+0.j     0.    +0.7071j 0.    +0.j    ]
 [0.    +0.j     0.    +0.7071j 0.7071+0.j     0.    +0.j    ]
 [0.    +0.j     0.    +0.j     0.    +0.j     1.    +0.j    ]]
generalized_BS
 [[ 1.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j -0.7071-0.j  0.7071+0.j  0.    +0.j]
 [ 0.    +0.j  0.7071+0.j  0.7071+0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.    +0.j -1.    -0.j]]
Corresponding Basis terms:
0H1V_A - 1 - [-0.70710678-0.j]
1H0V_A - 2 - [0.70710678+0.j]
